# Replicando el paper RFsp

## Pag. 13: Meuse data set, regression, 2D, no covariates
Compare performance of geostatistical model of *geoR* with RFsp model of *ranger*.

Assume that concentration of Zn in soil is controled by the behaviour of the river.

In [1]:
library(sp)

In [2]:
demo(meuse, echo=FALSE)

Warning message in showSRID(uprojargs, format = "PROJ", multiline = "NO", prefer_proj = prefer_proj):
"Discarded datum Amersfoort in Proj4 definition"


### 1. Ordinary Kriging (OK)

In [3]:
library(geoR)

--------------------------------------------------------------
 Analysis of Geostatistical Data
 For an Introduction to geoR go to http://www.leg.ufpr.br/geoR
 geoR version 1.8-1 (built on 2020-02-08) is now loaded
--------------------------------------------------------------




__meuse__ contains the coordinates for 164 locations along with a set of covariates. Covariates include metals concentration, elevation, distance, and some others that I don't recognise.

In [4]:
colnames(data.frame(meuse))

[1] "x"        "y"        "cadmium"  "copper"   "lead"     "zinc"    
 [7] "elev"     "dist"     "om"       "ffreq"    "soil"     "lime"    
[13] "landuse"  "dist.m"   "optional"

*as.geodata* from *geoR* converts the matrix to a geodata object made up of a list with 2 components: coordinates, and data.

In [5]:
zinc_geo = as.geodata(meuse['zinc'])
zinc_geo

$coords
         x      y
1   181072 333611
2   181025 333558
3   181165 333537
4   181298 333484
5   181307 333330
6   181390 333260
7   181165 333370
8   181027 333363
9   181060 333231
10  181232 333168
11  181191 333115
12  181032 333031
13  180874 333339
14  180969 333252
15  181011 333161
16  180830 333246
17  180763 333104
18  180694 332972
19  180625 332847
20  180555 332707
21  180642 332708
22  180704 332717
23  180704 332664
24  181153 332925
25  181147 332823
26  181167 332778
27  181008 332777
28  180973 332687
29  180916 332753
30  181352 332946
31  181133 332570
32  180878 332489
33  180829 332450
34  180954 332399
35  180956 332318
37  180710 332330
38  180632 332445
39  180530 332538
40  180478 332578
41  180383 332476
42  180494 332330
43  180561 332193
44  180451 332175
45  180410 332031
46  180355 332299
47  180292 332157
48  180283 332014
49  180282 331861
50  180270 331707
51  180199 331591
52  180135 331552
53  180237 332351
54  180103 332297
55  179973 332255
56

Compute variance of natural log of the data:

In [6]:
ini_v = c(var(log1p(zinc_geo$data)), 500)
ini_v

[1]   0.5181181 500.0000000

Estimate exponential variogram usign ML/REML, for a Box-Cox log-transformation of the data:

In [7]:
zinc_vgm = likfit(zinc_geo, lambda=0, ini=ini_v, cov.model='exponential')
zinc_vgm

kappa not used for the exponential correlation function
---------------------------------------------------------------
likfit: likelihood maximisation using the function optim.
likfit: Use control() to pass additional
         arguments for the maximisation function.
        For further details see documentation for optim.
likfit: It is highly advisable to run this function several
        times with different initial values for the parameters.
likfit: WARNING: This step can be time demanding!
---------------------------------------------------------------
likfit: end of numerical maximisation.


likfit: estimated model parameters:
      beta      tausq    sigmasq        phi 
"  6.1553" "  0.0164" "  0.5928" "500.0001" 
Practical Range with cor=0.05 for asymptotic range: 1497.866

likfit: maximised log-likelihood = -1014

Generate predictions with conventional kriging using the previously calculated variogram:

In [8]:
# Prediction locations
locs = meuse.grid@coords
zinc_ok = krige.conv(zinc_geo, locations=locs, krige=krige.control(obj.m=zinc_vgm))

krige.conv: model with constant mean
krige.conv: performing the Box-Cox data transformation
krige.conv: back-transforming the predicted mean and variance
krige.conv: Kriging performed using global neighbourhood 


### 2. Random Forest (spatial)

#### 2.1 Only buffer distance

We add the geographical distance to observation points as a covariate. First, we derive buffer distances for each individual point using the *buffer* function in the *raster* package.

In [9]:
library(raster)

In [10]:
grid_dist0 = GSIF::buffer.dist(meuse['zinc'], meuse.grid[1], 
                              as.factor(1:nrow(meuse)))

Warning message in showSRID(uprojargs, format = "PROJ", multiline = "NO", prefer_proj = prefer_proj):
"Discarded datum Unknown based on Bessel 1841 ellipsoid in Proj4 definition"


We have generated a gridded map for each observation point.

Now define the model that predicts the Zn concentration as a function of the buffer distances:

In [11]:
dn0 = paste(names(grid_dist0), collapse='+')
formula0 = as.formula(paste('zinc ~ ', dn0))
formula0

zinc ~ layer.1 + layer.2 + layer.3 + layer.4 + layer.5 + layer.6 + 
    layer.7 + layer.8 + layer.9 + layer.10 + layer.11 + layer.12 + 
    layer.13 + layer.14 + layer.15 + layer.16 + layer.17 + layer.18 + 
    layer.19 + layer.20 + layer.21 + layer.22 + layer.23 + layer.24 + 
    layer.25 + layer.26 + layer.27 + layer.28 + layer.29 + layer.30 + 
    layer.31 + layer.32 + layer.33 + layer.34 + layer.35 + layer.36 + 
    layer.37 + layer.38 + layer.39 + layer.40 + layer.41 + layer.42 + 
    layer.43 + layer.44 + layer.45 + layer.46 + layer.47 + layer.48 + 
    layer.49 + layer.50 + layer.51 + layer.52 + layer.53 + layer.54 + 
    layer.55 + layer.56 + layer.57 + layer.58 + layer.59 + layer.60 + 
    layer.61 + layer.62 + layer.63 + layer.64 + layer.65 + layer.66 + 
    layer.67 + layer.68 + layer.69 + layer.70 + layer.71 + layer.72 + 
    layer.73 + layer.74 + layer.75 + layer.76 + layer.77 + layer.78 + 
    layer.79 + layer.80 + layer.81 + layer.82 + layer.83 + layer.84 + 
    layer.85

The zinc is nor modelled as a function of 155 layers. 

Then create a regression matrix: overlaying points and covariates (layers). 

__NOTE:__ No entendi bien.

In [12]:
overlay_zinc = over(meuse['zinc'], grid_dist0)

In [13]:
regm_zinc = cbind(meuse@data['zinc'], overlay_zinc)
head(regm_zinc, 2)

,zinc,layer.1,layer.2,layer.3,layer.4,layer.5,layer.6,layer.7,layer.8,layer.9,...,layer.146,layer.147,layer.148,layer.149,layer.150,layer.151,layer.152,layer.153,layer.154,layer.155
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1022,0.00000,89.44272,144.222,268.3282,368.7818,481.6638,268.3282,243.3105,400.0000,...,4313.514,4383.606,4431.523,4186.263,4068.415,3920.204,3855.386,3982.763,3613.530,3468.025
2,1141,89.44272,0.00000,160.000,282.8427,344.0930,456.0702,226.2742,160.0000,322.4903,...,4224.121,4294.182,4342.350,4096.828,3979.146,3830.822,3766.165,3893.995,3524.089,3383.726


Finally, fit the model using *ranger* for quantile regression forests.

In [14]:
library(ranger)
model_zinc = ranger(formula0, regm_zinc, quantreg=TRUE, num.trees=150,
                   min.node.size=4, mtry=98)
model_zinc

Ranger result

Call:
 ranger(formula0, regm_zinc, quantreg = TRUE, num.trees = 150,      min.node.size = 4, mtry = 98) 

Type:                             Regression 
Number of trees:                  150 
Sample size:                      155 
Number of independent variables:  155 
Mtry:                             98 
Target node size:                 4 
Variable importance mode:         none 
Splitrule:                        variance 
OOB prediction error (MSE):       61452.66 
R squared (OOB):                  0.5439275 

The buffer discances explain 50% of the variability in the data.

zinc_rfd = predict(model_zinc, grid_dist0@data)
zinc_rfd

#### 2.2 Add thematic covariates

Now add covariates to the model, like global surface water occurrence and digital elevation model.

In [18]:
library(rgdal)

In [29]:
meuse.grid$SW0 = readGDAL('Meuse_GlobalSurfaceWater_occurrence.tif')$band1[meuse.grid@grid.index]

Meuse_GlobalSurfaceWater_occurrence.tif has GDAL driver GTiff 
and has 104 rows and 78 columns


Warning message in getProjectionRef(x, OVERRIDE_PROJ_DATUM_WITH_TOWGS84 = OVERRIDE_PROJ_DATUM_WITH_TOWGS84, :
"Discarded datum Amersfoort in Proj4 definition: +proj=sterea +lat_0=52.1561605555556 +lon_0=5.38763888888889 +k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel +units=m +no_defs"
Warning message in showSRID(uprojargs, format = "PROJ", multiline = "NO", prefer_proj = prefer_proj):
"Discarded datum Unknown based on Bessel 1841 ellipsoid in Proj4 definition"


In [30]:
meuse.grid$AHN = readGDAL('ahn.asc')$band1[meuse.grid@grid.index]

ERROR: Error in .local(.Object, ...): 



In [17]:
library(GDAL)

ERROR: Error in library(GDAL): there is no package called 'GDAL'


### 3. OK vs. RFsp

## Others

Some tricks to work with packages:

In [ ]:
# sp, geoR, raster

In [3]:
#install.packages('raster', 
#                 'C:/Users/Guillermina/.conda/envs/tesis/Lib/R/library', 
#                 repos='http://cran.us.r-project.org')

package 'raster' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Guillermina\AppData\Local\Temp\RtmpQtDf3l\downloaded_packages


In [ ]:
library(rgdal)
help(check_sp_version)

In [ ]:
sessionInfo()

In [ ]:
Sys.getenv("R_LIBS_USER")